### 1. Создание корпуса

Корпус состоит из четырех статей про рок-музыку с сайта https://elibrary.ru/keywords.asp, поиск по выражению "рок-музыка". В качестве ключевых слов были взяты ключевые слова к статьям.

In [ ]:
start_keywords = {}
documents = {}
samples_folder = ""
for filenumber in range(1, 5):
  filepath = samples_folder + "t%d.txt"%filenumber
  with open(filepath, "r") as file:
    kwords, doc = file.read().split("\n\n")
    start_keywords["doc%d"%filenumber] = kwords.split(", ")
    documents["doc%d"%filenumber] = doc.replace("\n", " ")

### 2. Самостоятельная разметка ключевых слов, пересечение, эталонная разметка

Списки ключевых слов пересекаются, но исходный более полный, поэтому эталонная разметка будет объединением двух списков. 

In [ ]:
marked_kw = dict(
    doc1 = "рок-музыка, сценический образ, культура XX века, музыка XX века, музыка, культура, субкультура".split(", "), 
    doc2 = "рок-музыка, рок-культура, субкультура, молодежная культура, контркультура, культура, культура XX века".split(", "), 
    doc3 = "рок-музыка, рок-культура, музыкальная индустрия, социализация, история музыки, музыка, музыка XX века".split(", "), 
    doc4 = "рок-музыка, агрессия, агрессивное поведение, русский рок".split(", ")
    )

start_keywords

{'doc1': ['художественный образ',
  'рок-музыка',
  'семиотика субкультур',
  'типологизация',
  'сценический образ',
  'рок-музыкант',
  'культура',
  'культура XX века',
  'музыка',
  'музыка XX века'],
 'doc2': ['рок-культура',
  'рок-музыка',
  'рок',
  'рок-поэзия',
  'рокпроизведение',
  'субкультура',
  'контркультура',
  'социализация',
  'маргинальность'],
 'doc3': ['музыкальная социализация',
  'рок-социализация',
  'рок-музыка',
  'рок-культура',
  'рок-ресурсы',
  'рокиндустрия',
  'социальный протест'],
 'doc4': ['речевая агрессия', 'рок-музыка', 'русский рок']}

In [ ]:
keywords_ = {key: start_keywords[key] for key in start_keywords.keys()}

for doc_num in range(1, 5):
  name = "doc%d"%doc_num

  for kw in marked_kw[name]:
    if kw not in keywords_[name]:
      keywords_[name].append(kw)

### 3. Извлечение ключевых слов

In [ ]:
!pip install python-rake
!pip install nltk
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import RAKE
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pymorphy2
import re

stop = stopwords.words("russian")
tokenizer = RegexpTokenizer("\w+")
morph = pymorphy2.MorphAnalyzer()
word_pattern = re.compile(r"[\w|-]+")

def normalize_text(text):
  lemmas = []

  for word in word_tokenize(text.lower()):
    if word_pattern.fullmatch(word):
      lemma = morph.parse(word)[0].normal_form
      lemmas.append(lemma)

  return " ".join(lemmas)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#RAKE
from pprint import pprint
rake = RAKE.Rake(stop)
rake_keywords = {}
print("RAKE")
print()
for doc_num in range(1, 5):
  print("Keywords of %d doc"%doc_num)
  rake_keywords["doc%d"%doc_num] = rake.run(normalize_text(documents["doc%d"%doc_num]), maxWords = 3, minFrequency = 2)
  pprint(rake_keywords["doc%d"%doc_num])
  print()

RAKE

Keywords of 1 doc
[('сторона выразитель', 4.0),
 ('наш мнение', 4.0),
 ('аудитория', 2.25),
 ('формирование', 1.6666666666666667),
 ('др', 1.6666666666666667),
 ('отметить', 1.3333333333333333),
 ('единение', 1.0),
 ('ранее', 1.0),
 ('затем', 1.0),
 ('связь', 1.0),
 ('запад', 1.0),
 ('получить', 1.0)]

Keywords of 2 doc
[('рок-культура', 5.984126984126984),
 ('социокультурный феномен', 4.833333333333334),
 ('культура', 2.888888888888889),
 ('субкультура', 2.0),
 ('который', 1.8333333333333333),
 ('общество', 1.75),
 ('аудитория', 1.75),
 ('направление', 1.3333333333333333),
 ('отношение', 1.0),
 ('время', 1.0),
 ('запад', 1.0),
 ('граница', 1.0),
 ('каждый', 1.0),
 ('целое', 1.0)]

Keywords of 3 doc
[('который', 2.2), ('общественный', 1.0)]

Keywords of 4 doc
[('текст рок-музыка', 8.266666666666666),
 ('текст', 2.2666666666666666),
 ('отличие', 1.6666666666666667),
 ('который', 1.5),
 ('связать', 1.0),
 ('случай', 1.0),
 ('связь', 1.0),
 ('зависимость', 1.0),
 ('связанный', 1.0)]

In [ ]:
#TextRank
!pip install summa

from summa import keywords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 3.0 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=2d4f74531e23c502cbf3c26a526483e8fee3ced08b2f21cd152cd4c5887ee69a
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa


In [ ]:
textrank_keywords = {}
print("TextRank")
print()
for doc_num in range(1, 5):
  print("Keywords of %d doc"%doc_num)
  textrank_keywords["doc%d"%doc_num] = keywords.keywords(
                              normalize_text(documents["doc%d"%doc_num]), 
                              language='russian', 
                              additional_stopwords=stop, scores=True
                                                        )
  pprint(textrank_keywords["doc%d"%doc_num])
  print()

TextRank

Keywords of 1 doc
[('образ', 0.32957821392117725),
 ('рок', 0.2624795216690377),
 ('the', 0.22470557649922265),
 ('костюм', 0.18173894952255198),
 ('сценический', 0.15947933480601317),
 ('культура', 0.1423907782804822),
 ('художественно', 0.14091184605146628),
 ('идея', 0.11232598279946449),
 ('определённый', 0.1050903234597864),
 ('свой художественный язык', 0.10391260408407589),
 ('яркий', 0.10386359633402503),
 ('знак', 0.09547984465932796),
 ('развитие', 0.09502437294944678),
 ('стиль', 0.08699167332727675),
 ('тип', 0.08693337756750104),
 ('современный', 0.08662642077383106),
 ('современность', 0.08662642077383106),
 ('своеобразный', 0.08371510889451475),
 ('элемент', 0.08093820653283557),
 ('стать', 0.08050913636711526),
 ('это особенность', 0.07736920642031621),
 ('имидж исполнитель', 0.07637274405772448),
 ('одежда', 0.07524261201337577),
 ('система', 0.07403621300939603),
 ('исследование', 0.07328530341380733),
 ('воин субкультурный', 0.07240528848296598),
 ('beatles

In [ ]:
# keybert
# !pip install keybert
from keybert import KeyBERT

kw_model = KeyBERT('clips/mfaq')
keybert_keywords = {}
print("KeyBERT")
print()
for doc_num in range(1, 5):
  print("Keywords of %d doc"%doc_num)
  keybert_keywords["doc%d"%doc_num] = kw_model.extract_keywords(
                                normalize_text(documents["doc%d"%doc_num]), 
                                keyphrase_ngram_range=(1, 2), 
                                stop_words=stop
                                                              )
  pprint(keybert_keywords["doc%d"%doc_num])
  print()

KeyBERT

Keywords of 1 doc
[('семантика образ', 0.973),
 ('эволюция сценический', 0.9713),
 ('музыкант образ', 0.9702),
 ('сценический образ', 0.9701),
 ('описать эволюция', 0.9701)]

Keywords of 2 doc
[('роккультура который', 0.9709),
 ('рокмузыка рок', 0.9695),
 ('роккультура', 0.9694),
 ('рассматривать рокмузыка', 0.9694),
 ('роккультура рассматривать', 0.9693)]

Keywords of 3 doc
[('рок социализация', 0.9715),
 ('музыкальный социализация', 0.9706),
 ('социальный процесс', 0.9706),
 ('поэзия идеология', 0.9702),
 ('социальный жизнь', 0.9702)]

Keywords of 4 doc
[('текст песня', 0.9675),
 ('определение речевой', 0.9674),
 ('магический текст', 0.9671),
 ('рок текст', 0.967),
 ('текст субкультура', 0.9669)]



### 4. Фильтрация текстов и выражений

In [ ]:
patterns = ("NOUN NOUN", "ADJF NOUN", "ADJS NOUN", "NOUN ROMN NOUN", "NOUN")

def pos(word):
  if morph.parse(word)[0].tag.POS is None:
    return ""
  return morph.parse(word)[0].tag.POS

def match_kword_pattern(kwords, patterns):
  
  filtered_kwords = []

  for kword in kwords:
    kword_words = kword[0].split()
    pattern = " ".join([pos(word) for word in kword_words])
    if pattern in patterns:
      filtered_kwords.append(kword)
  
  return filtered_kwords

def filter_kword_results(keywords):
  filtered_keywords = {}

  for doc in keywords.keys():
    filtered_keywords[doc] = match_kword_pattern(keywords[doc], patterns)

  return filtered_keywords

In [ ]:
filtered_rake_keywords = filter_kword_results(rake_keywords)
filtered_textrank_keywords = filter_kword_results(textrank_keywords)
filtered_keybert_keywords = filter_kword_results(keybert_keywords)

### 5. Оценка качества

In [ ]:
def quality_measure(predicts, targets):
  predicts = list(map(lambda x: x[0], predicts))
  targets = list(map(lambda x: normalize_text(x), targets))
  tp, fp, fn = 0, 0, 0

  for predict in predicts:
    if predict in targets:
      tp += 1
    else:
      fp += 1
  
  for target in targets:
    if target not in predicts:
      fn += 1

  precision = tp/(tp+fp) 
  recall = tp/(tp+fn)
  
  if precision+recall == 0:
    f_score = 0
  else:
    f_score = 2*precision*recall/(precision + recall)

  return {"precision": precision,
          "recall": recall,
          "f-score": f_score}

#rake_keywords["doc1"], keywords_["doc1"]

In [ ]:
print("RAKE algoritm: ", quality_measure(rake_keywords["doc1"], keywords_["doc1"]))
print("RAKE algoritm + filter: ", quality_measure(filtered_rake_keywords["doc1"], keywords_["doc1"]))

RAKE algoritm:  {'precision': 0.0, 'recall': 0.0, 'f-score': 0}
RAKE algoritm + filter:  {'precision': 0.0, 'recall': 0.0, 'f-score': 0}


In [ ]:
print("TextRank algoritm: ", quality_measure(textrank_keywords["doc1"], keywords_["doc1"]))
print("TextRank algoritm + filter: ", quality_measure(filtered_textrank_keywords["doc1"], keywords_["doc1"]))

TextRank algoritm:  {'precision': 0.025423728813559324, 'recall': 0.2727272727272727, 'f-score': 0.04651162790697675}
TextRank algoritm + filter:  {'precision': 0.05660377358490566, 'recall': 0.2727272727272727, 'f-score': 0.09375}


In [ ]:
print("KeyBERT algoritm: ", quality_measure(keybert_keywords["doc1"], keywords_["doc1"]))
print("KeyBERT algoritm + filter: ", quality_measure(filtered_keybert_keywords["doc1"], keywords_["doc1"]))

KeyBERT algoritm:  {'precision': 0.2, 'recall': 0.09090909090909091, 'f-score': 0.12500000000000003}
KeyBERT algoritm + filter:  {'precision': 0.3333333333333333, 'recall': 0.09090909090909091, 'f-score': 0.14285714285714288}
